In [ ]:
import sys
from pathlib import Path
import os
import json
import pickle

# Determine the current working directory of the notebook
notebook_dir = Path(os.getcwd())

# Add the parent directory to sys.path
parent_dir = notebook_dir.parent
sys.path.append(str(parent_dir))

# Now you can import aux.py
from aux import connect_to_db
import pandas as pd
import psycopg2
from flask import jsonify
from psycopg2.extras import RealDictCursor
from db_values import HOST, DBNAME, USER, PASSWORD

In [64]:
conn = psycopg2.connect(
            dbname=DBNAME,  
            user=USER,         
            password=PASSWORD,     
            host="localhost",             
        )

In [65]:
sql_query = "SELECT * FROM players"  
sql_data = pd.read_sql(sql_query, conn)
sql_data.head()

/tmp/ipykernel_3831/3933262908.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sql_data = pd.read_sql(sql_query, conn)


,player_id,name
0,1,Popyrin A.
1,2,Shevchenko A.
2,3,Safiullin R.
3,4,Rune H.
4,5,Dimitrov G.


In [66]:
csv_data = pd.read_csv("../data/players_data.csv")  # Columns: 'name', 'other_data'
csv_data.head()

,rank,name,age,points,nationality,atp_code
0,1,Jannik Sinner,23,11830,ITA,s0ag
1,2,Alexander Zverev,27,7915,GER,z355
2,3,Carlos Alcaraz,21,7010,ESP,a0e2
3,4,Taylor Fritz,27,5100,USA,fb98
4,5,Daniil Medvedev,28,5030,RUS,mm58


In [67]:
# Preprocess CSV names
csv_data['first_name'] = csv_data['name'].apply(lambda x: x.split()[0])  # First word
csv_data['last_name'] = csv_data['name'].apply(lambda x: x.split()[-1])  # Last word
csv_data['key'] = csv_data['last_name'] + " " + csv_data['first_name'].str[0]  # 'LastName FirstInitial'

# Preprocess SQL names
sql_data['last_name'] = sql_data['name'].apply(lambda x: x.split()[0])  # First word
sql_data['initial'] = sql_data['name'].apply(lambda x: x.split()[-1].strip('.'))  # Last character
sql_data['key'] = sql_data['last_name'] + " " + sql_data['initial']  # 'LastName Initial'

In [68]:
csv_data.head()

,rank,name,age,points,nationality,atp_code,first_name,last_name,key
0,1,Jannik Sinner,23,11830,ITA,s0ag,Jannik,Sinner,Sinner J
1,2,Alexander Zverev,27,7915,GER,z355,Alexander,Zverev,Zverev A
2,3,Carlos Alcaraz,21,7010,ESP,a0e2,Carlos,Alcaraz,Alcaraz C
3,4,Taylor Fritz,27,5100,USA,fb98,Taylor,Fritz,Fritz T
4,5,Daniil Medvedev,28,5030,RUS,mm58,Daniil,Medvedev,Medvedev D


In [69]:
sql_data.head()

,player_id,name,last_name,initial,key
0,1,Popyrin A.,Popyrin,A,Popyrin A
1,2,Shevchenko A.,Shevchenko,A,Shevchenko A
2,3,Safiullin R.,Safiullin,R,Safiullin R
3,4,Rune H.,Rune,H,Rune H
4,5,Dimitrov G.,Dimitrov,G,Dimitrov G


In [71]:
# Merge CSV and SQL data on the key
merged_data = pd.merge(
    sql_data, csv_data, on='key', how='left', suffixes=('_sql', '_csv')
)
merged_data.head()

,player_id,name_sql,last_name_sql,initial,key,rank,name_csv,age,points,nationality,atp_code,first_name,last_name_csv
0,1,Popyrin A.,Popyrin,A,Popyrin A,24.0,Alexei Popyrin,25.0,1865.0,AUS,p09z,Alexei,Popyrin
1,2,Shevchenko A.,Shevchenko,A,Shevchenko A,78.0,Alexander Shevchenko,23.0,715.0,KAZ,s0h2,Alexander,Shevchenko
2,3,Safiullin R.,Safiullin,R,Safiullin R,60.0,Roman Safiullin,27.0,923.0,RUS,sx50,Roman,Safiullin
3,4,Rune H.,Rune,H,Rune H,13.0,Holger Rune,21.0,3025.0,DEN,r0dg,Holger,Rune
4,5,Dimitrov G.,Dimitrov,G,Dimitrov G,10.0,Grigor Dimitrov,33.0,3350.0,BUL,d875,Grigor,Dimitrov


In [72]:
# Identify unmatched rows
unmatched = merged_data[merged_data['name_csv'].isna()]

In [73]:
len(unmatched)

42

## Fuzz Approach

In [74]:
# Fuzz
from rapidfuzz import fuzz, process

In [75]:
# Create a mapping table
mapping = []

for sql_key in sql_data['key']:
    match = process.extractOne(
        sql_key, 
        csv_data['key'], 
        scorer=fuzz.ratio
    )
    if match and match[1] > 80:  # 80% similarity threshold
        mapping.append({
            'sql_name': sql_key,
            'csv_name': match[0],
            'similarity': match[1]
        })

mapping_df = pd.DataFrame(mapping)

In [76]:
mapping_df.sort_values("similarity").head(20)

,sql_name,csv_name,similarity
122,Kwon S.W,Kwon S,85.714286
127,Wolf J.J,Wolf J,85.714286
169,Tseng H,Tseng C,85.714286
51,Galan D.E,Galan D,87.500000
191,Mccabe J,McCabe J,87.500000
251,Tseng C.H,Tseng C,87.500000
223,Sanchez N,Sanchez D,88.888889
22,Struff J.L,Struff J,88.888889
23,Huesler M.A,Huesler M,90.000000
146,Mcdonald M,McDonald M,90.000000


In [77]:
unmatched_sql = sql_data[~sql_data['key'].isin(mapping_df['sql_name'])]
len(unmatched_sql)

26

## Conclusion

With the Fuzz Approach we reduced the incompatibilities between players, e.g. Huesler M.A	and Huesler M	

Let's manually search the others

In [ ]:
safe_map_sql_csv = {
    "Van De Zandschulp B.": "Botic van de Zandschulp",
    "Bautista Agut R.": "Roberto Bautista Agut",
    "O Connell C.": "Christopher O'Connell",
    "Van Assche L.": "Luca Van Assche",
    "Carballes Baena R.": "Roberto Carballes Baena",
    "De Jong J.": "Jesper de Jong",
}

In [131]:
chat_generated_map = {
    "Van De Zandschulp B.": "Botic van de Zandschulp",
    "Bautista Agut R.": "Roberto Bautista Agut",
    "O Connell C.": "Christopher O'Connell",
    "Van Assche L.": "Luca Van Assche",
    "Carballes Baena R.": "Roberto Carballes Baena",
    "De Jong J.": "Jesper de Jong",
    "Davidovich Fokina A.": "Alejandro Davidovich Fokina",
    "De Minaur A.": "Alex de Minaur",
    "Llamas Ruiz P.": "Pablo Llamas Ruiz",
    "Diaz Acosta F.": "Facundo Diaz Acosta",
    "Zapata Miralles B.": "Bernabe Zapata Miralles",
    "Ugo Carabelli C.": "Camilo Ugo Carabelli",
    "Moreno De Alboran N.": "Nicolas Moreno De Alboran",
    #"Murray A.": "Andy Murray",
    "Barrios M.": "Tomas Barrios Vera",
    "Seyboth Wild T.": "Thiago Seyboth Wild",
    "Meligeni Alves F.": "Felipe Meligeni Alves",
    "Jorda Sanchis D.": "David Jorda Sanchis",
    "Moro Canas A.": "Alejandro Moro Canas",
    "Mpetshi G.": "Giovanni Mpetshi Perricard",
    "Carreno Busta P.": "Pablo Carreno Busta",
    #"Johnson S.": "Steve Johnson",
    "Pacheco Mendez R.": "Rodrigo Pacheco Mendez",
    "Barranco Cosano J.": "Javier Barranco Cosano",
    #"Van Rijthoven T.": "Tim van Rijthoven",
    #"Forbes M.": "Maxime Forbes"
}

In [132]:
[player for player in chat_generated_map.values() if player not in csv_data["name"].values]

[]

### Create Full Dictionary

In [117]:
mapping_df

,sql_name,csv_name,similarity
0,Popyrin A,Popyrin A,100.0
1,Shevchenko A,Shevchenko A,100.0
2,Safiullin R,Safiullin R,100.0
3,Rune H,Rune H,100.0
4,Dimitrov G,Dimitrov G,100.0
...,...,...,...
251,Tseng C.H,Tseng C,87.5
252,Dzumhur D,Dzumhur D,100.0
253,Gerasimov E,Gerasimov E,100.0
254,Ramanathan R,Ramanathan R,100.0


In [126]:
merged_data = mapping_df.merge(
    sql_data[["name", "key"]], 
    left_on="sql_name", 
    right_on="key", 
    how="left",
    suffixes=('_map', '_sql')
)
merged_data

,sql_name,csv_name,similarity,name,key
0,Popyrin A,Popyrin A,100.0,Popyrin A.,Popyrin A
1,Shevchenko A,Shevchenko A,100.0,Shevchenko A.,Shevchenko A
2,Safiullin R,Safiullin R,100.0,Safiullin R.,Safiullin R
3,Rune H,Rune H,100.0,Rune H.,Rune H
4,Dimitrov G,Dimitrov G,100.0,Dimitrov G.,Dimitrov G
...,...,...,...,...,...
251,Tseng C.H,Tseng C,87.5,Tseng C.H.,Tseng C.H
252,Dzumhur D,Dzumhur D,100.0,Dzumhur D.,Dzumhur D
253,Gerasimov E,Gerasimov E,100.0,Gerasimov E.,Gerasimov E
254,Ramanathan R,Ramanathan R,100.0,Ramanathan R.,Ramanathan R


In [127]:
final_merged_data = merged_data.merge(
    csv_data[["name", "key"]], 
    left_on="csv_name", 
    right_on="key", 
    how="left",
    suffixes=('_map', '_csv')
)

In [128]:
final_merged_data

,sql_name,csv_name,similarity,name_map,key_map,name_csv,key_csv
0,Popyrin A,Popyrin A,100.0,Popyrin A.,Popyrin A,Alexei Popyrin,Popyrin A
1,Shevchenko A,Shevchenko A,100.0,Shevchenko A.,Shevchenko A,Alexander Shevchenko,Shevchenko A
2,Safiullin R,Safiullin R,100.0,Safiullin R.,Safiullin R,Roman Safiullin,Safiullin R
3,Rune H,Rune H,100.0,Rune H.,Rune H,Holger Rune,Rune H
4,Dimitrov G,Dimitrov G,100.0,Dimitrov G.,Dimitrov G,Grigor Dimitrov,Dimitrov G
...,...,...,...,...,...,...,...
253,Tseng C.H,Tseng C,87.5,Tseng C.H.,Tseng C.H,Chun-Hsin Tseng,Tseng C
254,Dzumhur D,Dzumhur D,100.0,Dzumhur D.,Dzumhur D,Damir Dzumhur,Dzumhur D
255,Gerasimov E,Gerasimov E,100.0,Gerasimov E.,Gerasimov E,Egor Gerasimov,Gerasimov E
256,Ramanathan R,Ramanathan R,100.0,Ramanathan R.,Ramanathan R,Ramkumar Ramanathan,Ramanathan R


In [ ]:
name_mapping = dict(zip(final_merged_data["name_map"], final_merged_data["name_csv"]))

In [133]:
len(name_mapping)

256

In [134]:
name_mapping.update(chat_generated_map)

In [158]:
len(name_mapping)

278

In [156]:
import json

In [ ]:
file_path = '/workspaces/smart-tennis-bet/data/names_mapping.csv'

In [ ]:
mapping_df = pd.DataFrame(list(name_mapping.items()), columns=["sql_name", "csv_name"])
mapping_df.to_csv(file_path, index=False)